In [1]:
import torch

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('./dataset/credit-a.csv', header=None)

In [4]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,30.83,0.000,0,0,9,0,1.25,0,0,1,1,0,202,0.0,-1
1,1,58.67,4.460,0,0,8,1,3.04,0,0,6,1,0,43,560.0,-1
2,1,24.50,0.500,0,0,8,1,1.50,0,1,0,1,0,280,824.0,-1
3,0,27.83,1.540,0,0,9,0,3.75,0,0,5,0,0,100,3.0,-1
4,0,20.17,5.625,0,0,9,0,1.71,0,1,0,1,2,120,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,0,21.08,10.085,1,1,11,1,1.25,1,1,0,1,0,260,0.0,1
649,1,22.67,0.750,0,0,0,0,2.00,1,0,2,0,0,200,394.0,1
650,1,25.25,13.500,1,1,13,7,2.00,1,0,1,0,0,200,1.0,1
651,0,17.92,0.205,0,0,12,0,0.04,1,1,0,1,0,280,750.0,1


In [5]:
X = data.iloc[:, :-1]

In [6]:
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,30.83,0.000,0,0,9,0,1.25,0,0,1,1,0,202,0.0
1,1,58.67,4.460,0,0,8,1,3.04,0,0,6,1,0,43,560.0
2,1,24.50,0.500,0,0,8,1,1.50,0,1,0,1,0,280,824.0
3,0,27.83,1.540,0,0,9,0,3.75,0,0,5,0,0,100,3.0
4,0,20.17,5.625,0,0,9,0,1.71,0,1,0,1,2,120,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,0,21.08,10.085,1,1,11,1,1.25,1,1,0,1,0,260,0.0
649,1,22.67,0.750,0,0,0,0,2.00,1,0,2,0,0,200,394.0
650,1,25.25,13.500,1,1,13,7,2.00,1,0,1,0,0,200,1.0
651,0,17.92,0.205,0,0,12,0,0.04,1,1,0,1,0,280,750.0


In [7]:
Y = data.iloc[:, -1].replace(-1, 0)

In [8]:
Y.value_counts()

1    357
0    296
Name: 15, dtype: int64

In [9]:
X = torch.from_numpy(X.values).type(torch.FloatTensor)

In [10]:
X.shape

torch.Size([653, 15])

In [11]:
Y = torch.from_numpy(Y.values.reshape(-1, 1)).type(torch.FloatTensor)

In [12]:
Y.size()

torch.Size([653, 1])

In [13]:
from torch import nn

In [14]:
model = nn.Sequential(
                    nn.Linear(15, 1),
                    nn.Sigmoid()
)

In [15]:
model

Sequential(
  (0): Linear(in_features=15, out_features=1, bias=True)
  (1): Sigmoid()
)

In [16]:
loss_fn = nn.BCELoss()

In [17]:
opt = torch.optim.SGD(model.parameters(), lr=0.00001)

In [18]:
batches = 16
no_of_batches = 653//16

In [20]:
for epoch in range(1000):
    for batch in range(no_of_batches):
        start = batch*batches
        end = start + batches
        x = X[start: end]
        y = Y[start: end]
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()

In [25]:
model.state_dict()

OrderedDict([('0.weight',
              tensor([[-0.0050,  0.0186,  0.0190, -0.2182, -0.0371,  0.0434,  0.0031, -0.1751,
                       -0.0670,  0.0362, -0.1870,  0.0666,  0.0795,  0.0029, -0.0063]])),
             ('0.bias', tensor([0.2619]))])

### 计算正确率

In [32]:
((model(X).data.numpy() > 0.5).astype('int') == Y.numpy()).mean()

0.7243491577335375